<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/03_01_char_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tensorflow 2.x와 관련 패키지 임포트

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

### 모듈 임포트

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils  import to_categorical

### 학습 시퀀스(training sequence) 정의

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180


### 문자와 사전 정의

In [4]:
char_set = sorted(list(set(sentence)))
mapping = {w: i for i, w in enumerate(char_set)}
print ("CHARACTERS: ")
print (len(char_set))
print (char_set)
print ("DICTIONARY: ")
print (len(mapping))
print (mapping)

CHARACTERS: 
25
[' ', "'", ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'y']
DICTIONARY: 
25
{' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'w': 23, 'y': 24}


### 변수 설정

In [5]:
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # any arbitrary number

print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 25
num_classes : 25


### print_list( )

In [0]:
def print_list(_name,_x):
    print('Type of [%s] is [%s]' %(_name,type(_x)))
    print('Length of [%s] is %s' %(_name,len(_x)))
    print('%s[0] Looks like %s' %(_name,_x[0]))

### 학습용 시퀀스 생성

In [7]:
dataX = [] # 170 sequences
dataY = [] # 170 sequences
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [mapping[c] for c in x_str]  # x str to index
    y = [mapping[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x, y))

print('\n') 
print ((type(dataX)))
print ("dataX[0] = %s" % dataX[0])
print ("dataX[1] = %s" % dataX[1])
print ("dataX[168] = %s" % dataX[168])
print ("dataX[169] = %s" % dataX[169])

print('\n')
print ((type(dataY)))
print ("dataY[0] = %s" % dataY[0])
print ("dataY[1] = %s" % dataY[1])
print ("dataY[168] = %s" % dataY[168])
print ("dataY[169] = %s" % dataY[169])


[  0/180] [if you wan]=>[f you want]
          [12, 9, 0, 24, 17, 22, 0, 23, 4, 16]=>[9, 0, 24, 17, 22, 0, 23, 4, 16, 21]
[  1/180] [f you want]=>[ you want ]
          [9, 0, 24, 17, 22, 0, 23, 4, 16, 21]=>[0, 24, 17, 22, 0, 23, 4, 16, 21, 0]
[  2/180] [ you want ]=>[you want t]
          [0, 24, 17, 22, 0, 23, 4, 16, 21, 0]=>[24, 17, 22, 0, 23, 4, 16, 21, 0, 21]
[  3/180] [you want t]=>[ou want to]
          [24, 17, 22, 0, 23, 4, 16, 21, 0, 21]=>[17, 22, 0, 23, 4, 16, 21, 0, 21, 17]
[  4/180] [ou want to]=>[u want to ]
          [17, 22, 0, 23, 4, 16, 21, 0, 21, 17]=>[22, 0, 23, 4, 16, 21, 0, 21, 17, 0]


<class 'list'>
dataX[0] = [12, 9, 0, 24, 17, 22, 0, 23, 4, 16]
dataX[1] = [9, 0, 24, 17, 22, 0, 23, 4, 16, 21]
dataX[168] = [0, 17, 9, 0, 21, 11, 8, 0, 20, 8]
dataX[169] = [17, 9, 0, 21, 11, 8, 0, 20, 8, 4]


<class 'list'>
dataY[0] = [9, 0, 24, 17, 22, 0, 23, 4, 16, 21]
dataY[1] = [0, 24, 17, 22, 0, 23, 4, 16, 21, 0]
dataY[168] = [17, 9, 0, 21, 11, 8, 0, 20, 8, 4]
dataY[169] = [9,

### 학습용 시퀀스 데이타 리쉐이프(reshape) - 1

In [8]:
# separate into input and output
dataX = np.array(dataX)
print (dataX.shape) # (170, 10)

dataY = np.array(dataY)
print (dataY.shape) # (170, 10) 

X = dataX[:,:]      
print (X.shape) # (170, 10)

y = dataY[:,-1]
print (y.shape) # (170,)

(170, 10)
(170, 10)
(170, 10)
(170,)


### 학습용 시퀀스 데이타 리쉐이프(reshape) - 2

In [9]:
sequences = [to_categorical(x, num_classes = data_dim) for x in X]
print (sequences[0].shape)   # (10,25)
print (sequences[169].shape) # (10,25)

X = np.array(sequences)
print (X.shape) # (170, 10, 25)

print (y.shape) # (170,)
y = to_categorical(y, num_classes = data_dim)
print (y.shape) # (170, 25)
print (y[0])

(10, 25)
(10, 25)
(170, 10, 25)
(170,)
(170, 25)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0.]


### 모델 설계

In [10]:
# define model
model = Sequential()
model.add(LSTM(hidden_size, input_shape=(sequence_length, num_classes)))
model.add(Dense(data_dim, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                23040     
_________________________________________________________________
dense (Dense)                (None, 25)                1625      
Total params: 24,665
Trainable params: 24,665
Non-trainable params: 0
_________________________________________________________________
None


### 학습 셋업 및 학습

In [11]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
print (X.shape) # (170, 10, 25)
print (y.shape) # (170, 25)
model.fit(X, y, epochs=200, verbose=2)

(170, 10, 25)
(170, 25)
Train on 170 samples
Epoch 1/200
170/170 - 3s - loss: 3.2047 - accuracy: 0.0765
Epoch 2/200
170/170 - 0s - loss: 3.1750 - accuracy: 0.1118
Epoch 3/200
170/170 - 0s - loss: 3.1435 - accuracy: 0.1765
Epoch 4/200
170/170 - 0s - loss: 3.0928 - accuracy: 0.1941
Epoch 5/200
170/170 - 0s - loss: 3.0241 - accuracy: 0.1882
Epoch 6/200
170/170 - 0s - loss: 2.9141 - accuracy: 0.1882
Epoch 7/200
170/170 - 0s - loss: 2.8551 - accuracy: 0.1882
Epoch 8/200
170/170 - 0s - loss: 2.8353 - accuracy: 0.1882
Epoch 9/200
170/170 - 0s - loss: 2.8046 - accuracy: 0.1882
Epoch 10/200
170/170 - 0s - loss: 2.7944 - accuracy: 0.1882
Epoch 11/200
170/170 - 0s - loss: 2.7869 - accuracy: 0.1882
Epoch 12/200
170/170 - 0s - loss: 2.7717 - accuracy: 0.1882
Epoch 13/200
170/170 - 0s - loss: 2.7600 - accuracy: 0.1882
Epoch 14/200
170/170 - 0s - loss: 2.7510 - accuracy: 0.1882
Epoch 15/200
170/170 - 0s - loss: 2.7372 - accuracy: 0.1882
Epoch 16/200
170/170 - 0s - loss: 2.7297 - accuracy: 0.1882
Epoc

### 학습후 텍스트 생성 함수 만들기에 필요한 모듈 임포트

In [0]:
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 학습후 텍스트 생성 함수 만들기

In [0]:
# 텍스트 생성 함수
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		print (encoded)
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		print (encoded)
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		print (encoded.shape)
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		print (yhat)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

학습 Sentence : 

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180

In [16]:
# 테스트 트리거 1
print(generate_seq(model, mapping, 10, 'want to bu', 20))
# 테스트 트리거 2
#print(generate_seq(model, mapping, 10, 'collect wo', 20))
# 원문에 없는 트리거
#print(generate_seq(model, mapping, 10, 'rhther tae', 20))

[23, 4, 16, 21, 0, 21, 17, 0, 5, 22]
[[23  4 16 21  0 21 17  0  5 22]]
(1, 10, 25)
[12]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12]
[[ 4 16 21  0 21 17  0  5 22 12]]
(1, 10, 25)
[14]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14]
[[16 21  0 21 17  0  5 22 12 14]]
(1, 10, 25)
[7]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7]
[[21  0 21 17  0  5 22 12 14  7]]
(1, 10, 25)
[0]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7, 0]
[[ 0 21 17  0  5 22 12 14  7  0]]
(1, 10, 25)
[4]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7, 0, 4]
[[21 17  0  5 22 12 14  7  0  4]]
(1, 10, 25)
[0]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7, 0, 4, 0]
[[17  0  5 22 12 14  7  0  4  0]]
(1, 10, 25)
[20]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7, 0, 4, 0, 20]
[[ 0  5 22 12 14  7  0  4  0 20]]
(1, 10, 25)
[11]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7, 0, 4, 0, 20, 11]
[[ 5 22 12 14  7  0  4  0 20 11]]
(1, 10, 25)
[12]
[23, 4, 16, 21, 0, 21, 17, 0, 5, 22, 12, 14, 7, 0, 4, 0, 20, 11, 12]
[[22 12 14  7  0 